# Welcome to "ML4Recsys : Intro to content-based filtering" Notebook

In this notebook we will try recommend list of film based on one film that the user already watch, so the instruction is:

1. Read the data
2. Make the vector representation
3. Calculate the similarity betweenfilm based on the vector representation

## Read the data

In [43]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [44]:
df = df.loc[0:10,]
df

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...
5,Schindler's List,"Biography, Drama, History",Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...","In German-occupied Poland during World War II,..."
6,The Lord of the Rings: The Return of the King,"Adventure, Drama, Fantasy",Peter Jackson,"Noel Appleby, Ali Astin, Sean Astin, David Aston",Gandalf and Aragorn lead the World of Men agai...
7,Pulp Fiction,"Crime, Drama",Quentin Tarantino,"Tim Roth, Amanda Plummer, Laura Lovelace, John...","The lives of two mob hit men, a boxer, a gangs..."
8,Fight Club,Drama,David Fincher,"Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier","An insomniac office worker, looking for a way ..."
9,The Lord of the Rings: The Fellowship of the Ring,"Adventure, Drama, Fantasy",Peter Jackson,"Alan Howard, Noel Appleby, Sean Astin, Sala Baker",A meek Hobbit from the Shire and eight compani...


## Actorsnya biar gak kepisah oleh tanda ","


In [45]:
Actor = df["Actors"]
for i in range(len(Actor)):
    Actor[i] = Actor[i].replace(" ","").replace(",", " ")

df["Actors"] = Actor
df

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,TimRobbins MorganFreeman BobGunton WilliamSadler,Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,MarlonBrando AlPacino JamesCaan RichardS.Caste...,The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,AlPacino RobertDuvall DianeKeaton RobertDeNiro,The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,ChristianBale HeathLedger AaronEckhart Michael...,When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,MartinBalsam JohnFiedler LeeJ.Cobb E.G.Marshall,A jury holdout attempts to prevent a miscarria...
5,Schindler's List,"Biography, Drama, History",Steven Spielberg,LiamNeeson BenKingsley RalphFiennes CarolineGo...,"In German-occupied Poland during World War II,..."
6,The Lord of the Rings: The Return of the King,"Adventure, Drama, Fantasy",Peter Jackson,NoelAppleby AliAstin SeanAstin DavidAston,Gandalf and Aragorn lead the World of Men agai...
7,Pulp Fiction,"Crime, Drama",Quentin Tarantino,TimRoth AmandaPlummer LauraLovelace JohnTravolta,"The lives of two mob hit men, a boxer, a gangs..."
8,Fight Club,Drama,David Fincher,EdwardNorton BradPitt MeatLoaf ZachGrenier,"An insomniac office worker, looking for a way ..."
9,The Lord of the Rings: The Fellowship of the Ring,"Adventure, Drama, Fantasy",Peter Jackson,AlanHoward NoelAppleby SeanAstin SalaBaker,A meek Hobbit from the Shire and eight compani...


## Misahin kata yang ada di dataframe

In [46]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english') #cari tau kata yang harus dihilangkan

row = []
for i in range(len(df)-1):
    text = " ".join(df.loc[i,])
    row.append(text)

row
data = " ".join(row).split()
data = [x.replace(',', "") for x in data]
data = [x.replace('.', "") for x in data]
data = list(dict.fromkeys(data)) #hilangin duplikat
data = [x.lower() for x in data] #hilangin capital

for word in data:
    if word in stop_words:
        data.remove(word)
print(len(data))
data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
228


['shawshank',
 'redemption',
 'crime',
 'drama',
 'frank',
 'darabont',
 'timrobbins',
 'morganfreeman',
 'bobgunton',
 'williamsadler',
 'two',
 'imprisoned',
 'men',
 'bond',
 'number',
 'years',
 'finding',
 'solace',
 'eventual',
 'redemption',
 'acts',
 'common',
 'decency',
 'godfather',
 'francis',
 'ford',
 'coppola',
 'marlonbrando',
 'alpacino',
 'jamescaan',
 'richardscastellano',
 'aging',
 'patriarch',
 'organized',
 'crime',
 'dynasty',
 'transfers',
 'control',
 'clandestine',
 'empire',
 'reluctant',
 'son',
 'godfather:',
 'part',
 'ii',
 'robertduvall',
 'dianekeaton',
 'robertdeniro',
 'early',
 'life',
 'career',
 'vito',
 'corleone',
 '1920s',
 'new',
 'york',
 'portrayed',
 'michael',
 'expands',
 'tightens',
 'grip',
 'the',
 'family',
 'syndicate',
 'dark',
 'knight',
 'action',
 'christopher',
 'nolan',
 'christianbale',
 'heathledger',
 'aaroneckhart',
 'michaelcaine',
 'menace',
 'known',
 'joker',
 'emerges',
 'mysterious',
 'past',
 'wreaks',
 'havoc',
 'ch

## Hapus '.', ',' dan Ubah ke Huruf Kecil 

In [47]:
def cleaning_docs(docs):
    listWord = docs
    listWord = [x.replace(',', "") for x in listWord]
    listWord = [x.replace('.', "") for x in listWord]
    listWord = [x.lower() for x in listWord]
    return listWord

## Vector Representation

In [48]:
def tf(word, docs):
    '''
    input: word and documents

    output: term-frequency score
    '''
    docs = cleaning_docs(" ".join(docs).split())
    listWord = list(" ".join(docs).split())

    return listWord.count(word)/len(listWord)

def idf(df, word):
    '''
    input: all documents and words

    output: inverse-document-frequency score
    '''
    documentsContainWord = 0
    for i in df.index:
        list = cleaning_docs(" ".join(df.loc[i,]).split())
        if (word in list):
            documentsContainWord += 1

    tots = len(df)/documentsContainWord
    return np.log(tots)

def tfidf(tf,idf):
    '''
    input: tf and idf score

    output: tf-idf score
    '''
    return tf*idf

def build_vec_representation(x):
    '''
    input: word representation
    
    output: vector representation
    '''
    all = []
    for i in range(len(df)):
        row = []
        docs = df.loc[i, ]
        for word in x:
            term_frequency = tf(word, docs)
            inverse_doc_frequency = idf(df, word)
            tf_idf = tfidf(term_frequency, inverse_doc_frequency)
            row.append(tf_idf)
        all.append(row)
    
    result = pd.DataFrame(all, columns=x, index=df['Title'])

    return result

vec_representation = build_vec_representation(data)
vec_representation

,shawshank,redemption,crime,drama,frank,darabont,timrobbins,morganfreeman,bobgunton,williamsadler,two,imprisoned,men,bond,number,years,finding,solace,eventual,redemption,acts,common,decency,godfather,francis,ford,coppola,marlonbrando,alpacino,jamescaan,richardscastellano,aging,patriarch,organized,crime,dynasty,transfers,control,clandestine,empire,...,club,david,fincher,edwardnorton,bradpitt,meatloaf,zachgrenier,insomniac,office,worker,looking,way,change,crosses,paths,devil-may-care,soap,maker,forming,underground,club,evolves,something,much,fellowship,alanhoward,salabaker,meek,hobbit,shire,eight,companions,set,journey,destroy,powerful,save,middle,earth,sauron
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
The Shawshank Redemption,0.07993,0.113650,0.020205,0.0,0.07993,0.07993,0.07993,0.07993,0.07993,0.07993,0.056825,0.07993,0.033720,0.07993,0.07993,0.07993,0.07993,0.07993,0.07993,0.113650,0.07993,0.07993,0.07993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020205,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
The Godfather,0.00000,0.000000,0.041802,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.082686,0.058784,0.058784,0.058784,0.082686,0.058784,0.082686,0.082686,0.082686,0.082686,0.082686,0.041802,0.082686,0.082686,0.082686,0.082686,0.082686,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
The Godfather: Part II,0.00000,0.000000,0.029568,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.041579,0.041579,0.041579,0.000000,0.041579,0.000000,0.000000,0.000000,0.000000,0.000000,0.029568,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
The Dark Knight,0.00000,0.000000,0.011437,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011437,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12 Angry Men,0.00000,0.000000,0.020901,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.034883,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020901,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000

## Calculate the similarity

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
def cos_similarity(x):
  '''
  input: all film vector representation

  output: matrix of cosine similarity between each film
  '''
  return cosine_similarity(x,x)

cos_sim = cosine_similarity(vec_representation)
cos_sim

array([[1.00000000e+00, 1.41314843e-02, 1.20882456e-02, 6.11887646e-03,
        3.41464930e-02, 2.94183345e-04, 1.77265745e-02, 1.63372529e-01,
        7.37185180e-03, 1.10475002e-02, 0.00000000e+00],
       [1.41314843e-02, 1.00000000e+00, 1.59852508e-01, 1.45314056e-02,
        1.54616067e-02, 6.98640925e-04, 4.44497963e-03, 1.35924405e-02,
        0.00000000e+00, 4.84881873e-03, 0.00000000e+00],
       [1.20882456e-02, 1.59852508e-01, 1.00000000e+00, 1.36146067e-02,
        1.32260487e-02, 2.18928458e-02, 5.55902593e-03, 1.16271409e-02,
        1.70590253e-02, 6.06407932e-03, 0.00000000e+00],
       [6.11887646e-03, 1.45314056e-02, 1.36146067e-02, 1.00000000e+00,
        6.69481419e-03, 1.44077532e-03, 3.05157556e-02, 5.88547266e-03,
        5.62597145e-02, 6.43258510e-02, 0.00000000e+00],
       [3.41464930e-02, 1.54616067e-02, 1.32260487e-02, 6.69481419e-03,
        1.00000000e+00, 3.21873279e-04, 1.93950839e-02, 5.29350858e-02,
        1.61314510e-02, 2.19407708e-02, 0.00000000e+

## Lihat Data


In [50]:
cosine_matrix = pd.DataFrame(cos_sim, columns=df['Title'], index=df['Title'])
cosine_matrix

Title,The Shawshank Redemption,The Godfather,The Godfather: Part II,The Dark Knight,12 Angry Men,Schindler's List,The Lord of the Rings: The Return of the King,Pulp Fiction,Fight Club,The Lord of the Rings: The Fellowship of the Ring,Forrest Gump
Title,,,,,,,,,,,
The Shawshank Redemption,1.000000,0.014131,0.012088,0.006119,0.034146,0.000294,0.017727,0.163373,0.007372,0.011048,0.0
The Godfather,0.014131,1.000000,0.159853,0.014531,0.015462,0.000699,0.004445,0.013592,0.000000,0.004849,0.0
The Godfather: Part II,0.012088,0.159853,1.000000,0.013615,0.013226,0.021893,0.005559,0.011627,0.017059,0.006064,0.0
The Dark Knight,0.006119,0.014531,0.013615,1.000000,0.006695,0.001441,0.030516,0.005885,0.056260,0.064326,0.0
12 Angry Men,0.034146,0.015462,0.013226,0.006695,1.000000,0.000322,0.019395,0.052935,0.016131,0.021941,0.0
Schindler's List,0.000294,0.000699,0.021893,0.001441,0.000322,1.000000,0.023204,0.000283,0.000000,0.001943,0.0
The Lord of the Rings: The Return of the King,0.017727,0.004445,0.005559,0.030516,0.019395,0.023204,1.000000,0.017050,0.000000,0.270679,0.0
Pulp Fiction,0.163373,0.013592,0.011627,0.005885,0.052935,0.000283,0.017050,1.000000,0.021272,0.027951,0.0
Fight Club,0.007372,0.000000,0.017059,0.056260,0.016131,0.000000,0.000000,0.021272,1.000000,0.013910,0.0


## Recommend the film

In [51]:
def recommend_film(film, cosine_matrix, k):
    '''
    input : one film that the user already watched, the cosine matrix and integer k

    output : list of k film (the title) that similar with the input film
    '''
    list = [i for i in cosine_matrix.index]
    if film in list:
        list.remove(film)
    get_row = cosine_matrix.loc[film, list]
    get_row.sort_values(ascending=False, inplace=True)
    return get_row.head(k)

print(recommend_film("The Shawshank Redemption", cosine_matrix, 10))

Title
Pulp Fiction                                         0.163373
12 Angry Men                                         0.034146
The Lord of the Rings: The Return of the King        0.017727
The Godfather                                        0.014131
The Godfather: Part II                               0.012088
The Lord of the Rings: The Fellowship of the Ring    0.011048
Fight Club                                           0.007372
The Dark Knight                                      0.006119
Schindler's List                                     0.000294
Forrest Gump                                         0.000000
Name: The Shawshank Redemption, dtype: float64
